# Тестовое задание. Windy.app.

1. Есть две таблички. В первой – юзеры, пришедшие в феврале с Apple Search Ads, и их покупки. Во второй – косты по Apple Search Ads в разрезе кампаний. 

Задача: 
- Найти, как минимум, две кампании, в рамках которых происходило что-то странное, и попробовать объяснить причину. 
- Найти три наиболее прибыльные кампании и три наиболее убыточные.
- В разрезе US кампании проанализировать пользователей и найти какие-либо закономерности.



In [2]:
import pandas as pd

In [4]:
users_df = pd.read_csv("users_asa_feb_flitered.csv")
costs_df = pd.read_csv("costs_asa_campaign_filtered.csv")

In [6]:
users_df.head()

,user_id,lang,pro,product,price,sail,kite,wind,surf,fish,snow,other,source,campaign,install_date
0,FA64312F-273A-41AB-BB9C-DB43AFE5B760,de_PL,0,NaN,NaN,0,0,0,0,0,0,1,Apple Search Ads,PL Discovery,01.02.2023
1,3FF6E426-B5D5-4C8B-A915-9FB58464EA50,en_NZ,0,NaN,NaN,1,0,0,0,1,0,0,Apple Search Ads,NZ Common_,01.02.2023
2,9C82BE9F-0212-47EA-872F-AF7237D43064,en_CA,1,co.windyapp.pro_sub_year_50_notrial,50.0,1,0,0,0,1,0,0,Apple Search Ads,CA Common_,01.02.2023
3,750657AF-3BB4-457A-9BBA-BE691CA5A75C,en_AR,0,NaN,NaN,0,1,0,0,0,0,0,Apple Search Ads,AR Skag,01.02.2023
4,A417C7DE-229E-4BB3-8BF0-17FA6883535E,ru_UA,0,NaN,NaN,0,0,0,0,0,0,1,Apple Search Ads,UA SKAG,01.02.2023


In [7]:
costs_df.head()

,Date,Campaign,Cost
0,2023-02-01,US Common_,"89,22"
1,2023-02-01,NZ Common_,"21,54"
2,2023-02-01,DE Common_,"18,15"
3,2023-02-01,UK Common_,"11,21"
4,2023-02-01,FR Common_,"7,83"


In [55]:
costs_df['Date'] = pd.to_datetime(costs_df['Date'])
users_df['install_date'] = pd.to_datetime(users_df['install_date'], format='%d.%m.%Y')

costs_df['Cost'] = costs_df['Cost'].str.replace(',', '.').astype('float')

costs_df.rename({"Campaign": "campaign"}, axis=1, inplace=True)

In [62]:
campaign_revenue = users_df.groupby(['campaign'], as_index=False)['price'].sum().sort_values("price", ascending=False)

In [57]:
campaign_revenue

campaign
US Common_      8110.0
NZ Common_      1560.0
UK Common_       700.0
DE Common_       600.0
CL Common_       555.0
                 ...  
IN Discovery       0.0
IE SKAG            0.0
IE Discovery       0.0
ID SKAG            0.0
KW Discovery       0.0
Name: price, Length: 105, dtype: float64

In [64]:
campaign_cost = costs_df.groupby(['campaign'], as_index=False)['Cost'].sum().sort_values("Cost", ascending=False)

In [65]:
campaign_cost

,campaign,Cost
100,US Common_,3487.89
67,NZ Common_,1367.59
19,DE Common_,417.11
29,FR Common_,406.30
98,UK Common_,406.24
...,...,...
24,EC SKAG,0.86
76,PK SKAG,0.67
51,KH SKAG,0.59
9,AZ SKAG,0.44


In [67]:
camps_stats = campaign_cost.merge(campaign_revenue, how='left')

In [70]:
camps_stats.head()

,campaign,Cost,price
0,US Common_,3487.89,8110.0
1,NZ Common_,1367.59,1560.0
2,DE Common_,417.11,600.0
3,FR Common_,406.30,295.0
4,UK Common_,406.24,700.0


In [71]:
camps_stats['price'] = camps_stats['price'].fillna(0)

In [73]:
campaign_total_users = users_df.groupby(['campaign'], as_index=False)['user_id'].nunique().rename({"user_id": "users_cnt"}, axis=1)

In [76]:
campaign_pro_users = users_df.groupby(['campaign', 'user_id'], as_index=False)['pro'].max()
campaign_pro_users = campaign_pro_users.groupby(['campaign'], as_index=False)['pro'].sum().rename({"pro": "pro_users_cnt"}, axis=1)

In [78]:
camps_stats = camps_stats.merge(campaign_total_users, how='left').fillna(0)
camps_stats = camps_stats.merge(campaign_pro_users, how='left').fillna(0)

In [80]:
camps_stats = camps_stats.rename({"Cost": "cost"}, axis=1)
camps_stats = camps_stats.rename({"price": "revenue"}, axis=1)

In [81]:
camps_stats.head()

,campaign,cost,revenue,users_cnt,pro_users_cnt
0,US Common_,3487.89,8110.0,1219.0,93.0
1,NZ Common_,1367.59,1560.0,886.0,29.0
2,DE Common_,417.11,600.0,367.0,11.0
3,FR Common_,406.30,295.0,461.0,6.0
4,UK Common_,406.24,700.0,210.0,12.0


In [84]:
camps_stats['conversion_rate'] = (camps_stats['pro_users_cnt']/camps_stats['users_cnt']).fillna(0)

In [85]:
# рассчитать CPPU, Profit, CPI

In [87]:
camps_stats['profit'] = camps_stats['revenue'] - camps_stats['cost']

In [91]:
camps_stats['cpi'] = camps_stats['revenue']/camps_stats['cost'] * 100

In [92]:
camps_stats

,campaign,cost,revenue,users_cnt,pro_users_cnt,conversion_rate,profit,cpi
0,US Common_,3487.89,8110.0,1219.0,93.0,0.076292,4622.11,232.518801
1,NZ Common_,1367.59,1560.0,886.0,29.0,0.032731,192.41,114.069275
2,DE Common_,417.11,600.0,367.0,11.0,0.029973,182.89,143.846947
3,FR Common_,406.30,295.0,461.0,6.0,0.013015,-111.30,72.606448
4,UK Common_,406.24,700.0,210.0,12.0,0.057143,293.76,172.311934
...,...,...,...,...,...,...,...,...
102,EC SKAG,0.86,0.0,3.0,0.0,0.000000,-0.86,0.000000
103,PK SKAG,0.67,0.0,3.0,0.0,0.000000,-0.67,0.000000
104,KH SKAG,0.59,0.0,0.0,0.0,0.000000,-0.59,0.000000
105,AZ SKAG,0.44,0.0,3.0,0.0,0.000000,-0.44,0.000000


In [93]:
camps_stats[camps_stats['users_cnt'] == 0]

,campaign,cost,revenue,users_cnt,pro_users_cnt,conversion_rate,profit,cpi
99,MO Discovery,1.44,0.0,0.0,0.0,0.0,-1.44,0.0
104,KH SKAG,0.59,0.0,0.0,0.0,0.0,-0.59,0.0


По кампаниям `MO Discovery`	и `KH SKAG` нет установок пользователей. Тому может быть много причин, например, неправильно считаем пользователей этой кампании (то есть пропускаем пользователей), либо кампаний откручивается не на целевую аудиторию.

In [95]:
camps_stats.sort_values("cpi").head(3)

,campaign,cost,revenue,users_cnt,pro_users_cnt,conversion_rate,profit,cpi
106,MO SKAG,0.16,0.0,1.0,0.0,0.0,-0.16,0.0
65,UK_prod_page,11.18,0.0,1.0,0.0,0.0,-11.18,0.0
64,KH Discovery,11.29,0.0,5.0,0.0,0.0,-11.29,0.0
